In [ ]:
#| default_exp main

In [ ]:
#| export

from functools import partial

from bedmap.prepare_images import df_images_from_pattern
from bedmap.validate_images import validate_images
from bedmap.create_thumbnails import create_thumbnails_col
from bedmap.embed_images import create_embeddings_col
from bedmap.create_umap_layout import create_umap_col

{'thumbnail_size': 128, 'model_name': 'timm/vit_small_patch14_reg4_dinov2.lvd142m', 'umap_spec': {'umap_n_components': 10, 'umap_n_neighbors': 15, 'umap_min_dist': 0.1}, 'paths': {'image_dir': None, 'image_glob': None, 'thumbnail_dir': None}}


/home/willsa/git/bedmap-dev/.venv/lib/python3.12/site-packages/umap/__init__.py:36: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


## 2. All-steps pipeline

In [ ]:
#| export

def process_images_to_umap(cfg):
    steps = [validate_images,
        partial(create_thumbnails_col, thumb_size=cfg.thumb_opts.size),
    ]
    if cfg.embed_opts is not None:
        steps.append(partial(create_embeddings_col, embed_model=cfg.embed_opts.model))
    if isinstance(cfg.umap_specs, list):
        for spec in cfg.umap_specs:
            steps.append(partial(create_umap_col, umap_specs=spec))
    elif cfg.umap_specs:
        steps.append(partial(create_umap_col, umap_specs=cfg.umap_specs))

    return steps

In [ ]:

#| export

def run_recipe(cfg):
    df = df_images_from_pattern(cfg.pattern)
    for fn in process_images_to_umap(cfg):
        df = fn(df)
    return df

In [ ]:
#| hide

import nbdev
nbdev.nbdev_export()